In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
features = pd.read_csv('../Data/training_set_features.csv')
labels = pd.read_csv('../Data/training_set_labels.csv')

In [3]:
df = pd.merge(features, labels, on='respondent_id', how='inner')

In [4]:
df = df.drop(columns=['employment_occupation', 'employment_industry', 'health_insurance', 'respondent_id'])

In [5]:
seas_df = df.drop(columns=['h1n1_concern',
                          'h1n1_knowledge',
                          'doctor_recc_h1n1',
                          'opinion_h1n1_vacc_effective',
                          'opinion_h1n1_risk',
                          'opinion_h1n1_sick_from_vacc',
                          'h1n1_vaccine'])

h1n1_df = df.drop(columns=['doctor_recc_seasonal',
                          'opinion_seas_vacc_effective',
                          'opinion_seas_risk',
                          'opinion_seas_sick_from_vacc',
                          'seasonal_vaccine'])

In [6]:
categorical_columns = [
    'sex',
    'hhs_geo_region',
    'census_msa',
    'race',
    'age_group',
    'behavioral_face_mask',
    'behavioral_wash_hands',
    'behavioral_antiviral_meds',
    'behavioral_outside_home',
    'behavioral_large_gatherings',
    'behavioral_touch_face',
    'behavioral_avoidance',
    'health_worker',
    'child_under_6_months',
    'chronic_med_condition',
    'education',
    'marital_status',
    'employment_status',
    'rent_or_own',
    'doctor_recc_h1n1',
    'doctor_recc_seasonal',
    'income_poverty'
]

numerical_columns = [
    'household_children',
    'household_adults',
    'h1n1_concern',
    'h1n1_knowledge',
    'opinion_h1n1_risk',
    'opinion_h1n1_vacc_effective',
    'opinion_h1n1_sick_from_vacc',
    'opinion_seas_vacc_effective',
    'opinion_seas_risk',
    'opinion_seas_sick_from_vacc',
    
]

for column in categorical_columns:
    curr_col = df[column]
    df.loc[df[column] == 1, column] = 'Yes'
    df.loc[df[column] == 0, column] = 'No'



## Deal with NAs

In [7]:
((df.isnull().sum() / len(df)) * 100).sort_values()

seasonal_vaccine                0.000000
census_msa                      0.000000
hhs_geo_region                  0.000000
sex                             0.000000
race                            0.000000
age_group                       0.000000
h1n1_vaccine                    0.000000
behavioral_face_mask            0.071142
behavioral_wash_hands           0.157262
behavioral_antiviral_meds       0.265848
behavioral_outside_home         0.307036
behavioral_large_gatherings     0.325757
h1n1_concern                    0.344479
h1n1_knowledge                  0.434343
behavioral_touch_face           0.479275
behavioral_avoidance            0.778822
household_children              0.932340
household_adults                0.932340
opinion_h1n1_risk               1.452803
opinion_h1n1_vacc_effective     1.464036
opinion_h1n1_sick_from_vacc     1.479013
opinion_seas_vacc_effective     1.729884
opinion_seas_risk               1.924589
opinion_seas_sick_from_vacc     2.010709
health_worker   

In [8]:
for column in numerical_columns:
    df[column] = df[column].fillna(df[column].mean())

df['health_worker'] = df['health_worker'].fillna(0)
df['behavioral_face_mask'] = df['behavioral_face_mask'].fillna(0)
df['behavioral_wash_hands'] = df['behavioral_wash_hands'].fillna(0)
df['behavioral_antiviral_meds'] = df['behavioral_antiviral_meds'].fillna(0)
df['behavioral_outside_home'] = df['behavioral_outside_home'].fillna(0)
df['behavioral_large_gatherings'] = df['behavioral_large_gatherings'].fillna(0)
df['behavioral_touch_face'] = df['behavioral_touch_face'].fillna(0)
df['behavioral_avoidance'] = df['behavioral_avoidance'].fillna(0)
df['child_under_6_months'] = df['child_under_6_months'].fillna(0)
df['chronic_med_condition'] = df['chronic_med_condition'].fillna(0)
df['marital_status'] = df['marital_status'].fillna('Not Married')
df['rent_or_own'] = df['rent_or_own'].fillna('Rent')
df['education'] = df['education'].fillna('Some College')
df['employment_status'] = df['employment_status'].fillna('Employed')
df['doctor_recc_h1n1'] = df['doctor_recc_h1n1'].fillna(1)
df['doctor_recc_seasonal'] = df['doctor_recc_seasonal'].fillna(1)
df['income_poverty'] = df['income_poverty'].fillna('<= $75,000, Above Poverty')

## Initial Run

In [9]:
X = df.drop(columns=['h1n1_vaccine', 'seasonal_vaccine'])
y = df[['h1n1_vaccine', 'seasonal_vaccine']]
y_h1n1 = df[['h1n1_vaccine']]
y_seas = df[['seasonal_vaccine']]

#### Categorical

In [10]:
#Get Binary Data for Categorical Variables
cat_df = X[categorical_columns]

In [11]:
recat_df = pd.get_dummies(data=cat_df).columns

#### Numerical

In [12]:
num_df = X[numerical_columns]

In [13]:
from sklearn.preprocessing import StandardScaler

#Scale Numerical Data
scaler = StandardScaler()
scaled_num = scaler.fit_transform(num_df)
scaled_num_df = pd.DataFrame(scaled_num, index=num_df.index, columns=num_df.columns)

In [14]:
encoded_df = pd.concat([num_df, scaled_num_df], axis=1)

In [15]:
encoded_df

,household_children,household_adults,h1n1_concern,h1n1_knowledge,opinion_h1n1_risk,opinion_h1n1_vacc_effective,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,household_children,household_adults,h1n1_concern,h1n1_knowledge,opinion_h1n1_risk,opinion_h1n1_vacc_effective,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc
0,0.0,0.0,1.0,0.0,1.0,3.0,2.0,2.0,1.0,2.0,-0.578667,-1.182177,-0.680609,-2.046928,-1.052050,-0.850610,-0.264426,-1.880954,-1.253366,-0.089516
1,0.0,0.0,3.0,2.0,4.0,5.0,4.0,4.0,2.0,4.0,-0.578667,-1.182177,1.520279,1.195647,1.298784,1.149360,1.214180,-0.024126,-0.524309,1.426260
2,0.0,2.0,1.0,1.0,1.0,3.0,1.0,4.0,1.0,2.0,-0.578667,1.484891,-0.680609,-0.425641,-1.052050,-0.850610,-1.003729,-0.024126,-1.253366,-0.089516
3,0.0,0.0,1.0,1.0,3.0,3.0,5.0,5.0,4.0,1.0,-0.578667,-1.182177,-0.680609,-0.425641,0.515173,-0.850610,1.953484,0.904289,0.933803,-0.847404
4,0.0,1.0,2.0,1.0,3.0,3.0,2.0,3.0,1.0,4.0,-0.578667,0.151357,0.419835,-0.425641,0.515173,-0.850610,-0.264426,-0.952540,-1.253366,1.426260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,0.0,0.0,2.0,0.0,1.0,3.0,1.0,5.0,2.0,2.0,-0.578667,-1.182177,0.419835,-2.046928,-1.052050,-0.850610,-1.003729,0.904289,-0.524309,-0.089516
26703,0.0,1.0,1.0,2.0,2.0,4.0,2.0,5.0,1.0,1.0,-0.578667,0.151357,-0.680609,1.195647,-0.268439,0.149375,-0.264426,0.904289,-1.253366,-0.847404
26704,0.0,0.0,2.0,2.0,4.0,4.0,2.0,5.0,4.0,2.0,-0.578667,-1.182177,0.419835,1.195647,1.298784,0.149375,-0.264426,0.904289,0.933803,-0.089516
26705,0.0,1.0,1.0,1.0,1.0,3.0,2.0,2.0,1.0,2.0,-0.578667,0.151357,-0.680609,-0.425641,-1.052050,-0.850610,-0.264426,-1.880954,-1.253366,-0.089516


In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(encoded_df, y, test_size=0.3, random_state=42)

In [17]:
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
X = np.asarray(encoded_df)

# Neural Network

In [23]:
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(2, activation='relu', input_dim=20),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(2, activation='sigmoid')
])

In [24]:
model.compile(optimizer='adam', 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [25]:
history = model.fit(
    X_train,
    y_train,
    batch_size=200,
    epochs=5000,
    validation_data=(X_test, y_test)
)

Epoch 1/5000
94/94 [==============================] - 0s 2ms/step - loss: 0.4645 - accuracy: 0.3990 - val_loss: 0.4506 - val_accuracy: 0.2854
Epoch 2/5000
94/94 [==============================] - 0s 987us/step - loss: 0.4594 - accuracy: 0.2916 - val_loss: 0.4490 - val_accuracy: 0.2854
Epoch 3/5000
94/94 [==============================] - 0s 937us/step - loss: 0.4566 - accuracy: 0.2916 - val_loss: 0.4488 - val_accuracy: 0.2854
Epoch 4/5000
94/94 [==============================] - 0s 932us/step - loss: 0.4543 - accuracy: 0.2956 - val_loss: 0.4447 - val_accuracy: 0.2953
Epoch 5/5000
94/94 [==============================] - 0s 937us/step - loss: 0.4516 - accuracy: 0.3181 - val_loss: 0.4431 - val_accuracy: 0.3267
Epoch 6/5000
94/94 [==============================] - 0s 942us/step - loss: 0.4503 - accuracy: 0.3370 - val_loss: 0.4396 - val_accuracy: 0.3160
Epoch 7/5000
94/94 [==============================] - 0s 907us/step - loss: 0.4492 - accuracy: 0.3471 - val_loss: 0.4403 - val_accuracy: 0

KeyboardInterrupt: 

In [26]:
y_true = y
y_predicted = model.predict(X)
y_predicted_binary = np.where(y_predicted > 0.5, 1, 0)

In [37]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y, y_predicted)

ValueError: continuous-multioutput format is not supported

## Submission Data

In [28]:
test_data = pd.read_csv('../Data/test_set_features.csv')
df_full = test_data

In [29]:
df = df_full.drop(columns=['employment_occupation', 'employment_industry', 'health_insurance', 'respondent_id'])

categorical_columns = [
    'sex',
    'hhs_geo_region',
    'census_msa',
    'race',
    'age_group',
    'behavioral_face_mask',
    'behavioral_wash_hands',
    'behavioral_antiviral_meds',
    'behavioral_outside_home',
    'behavioral_large_gatherings',
    'behavioral_touch_face',
    'behavioral_avoidance',
    'health_worker',
    'child_under_6_months',
    'chronic_med_condition',
    'education',
    'marital_status',
    'employment_status',
    'rent_or_own',
    'doctor_recc_h1n1',
    'doctor_recc_seasonal',
    'income_poverty'
]

numerical_columns = [
    'household_children',
    'household_adults',
    'h1n1_concern',
    'h1n1_knowledge',
    'opinion_h1n1_risk',
    'opinion_h1n1_vacc_effective',
    'opinion_h1n1_sick_from_vacc',
    'opinion_seas_vacc_effective',
    'opinion_seas_risk',
    'opinion_seas_sick_from_vacc',
    
]

for column in categorical_columns:
    curr_col = df[column]
    df.loc[df[column] == 1, column] = 'Yes'
    df.loc[df[column] == 0, column] = 'No'

In [30]:
for column in numerical_columns:
    df[column] = df[column].fillna(df[column].mean())

df['health_worker'] = df['health_worker'].fillna(0)
df['behavioral_face_mask'] = df['behavioral_face_mask'].fillna(0)
df['behavioral_wash_hands'] = df['behavioral_wash_hands'].fillna(0)
df['behavioral_antiviral_meds'] = df['behavioral_antiviral_meds'].fillna(0)
df['behavioral_outside_home'] = df['behavioral_outside_home'].fillna(0)
df['behavioral_large_gatherings'] = df['behavioral_large_gatherings'].fillna(0)
df['behavioral_touch_face'] = df['behavioral_touch_face'].fillna(0)
df['behavioral_avoidance'] = df['behavioral_avoidance'].fillna(0)
df['child_under_6_months'] = df['child_under_6_months'].fillna(0)
df['chronic_med_condition'] = df['chronic_med_condition'].fillna(0)
df['marital_status'] = df['marital_status'].fillna('Not Married')
df['rent_or_own'] = df['rent_or_own'].fillna('Rent')
df['education'] = df['education'].fillna('Some College')
df['employment_status'] = df['employment_status'].fillna('Employed')
df['doctor_recc_h1n1'] = df['doctor_recc_h1n1'].fillna(1)
df['doctor_recc_seasonal'] = df['doctor_recc_seasonal'].fillna(1)
df['income_poverty'] = df['income_poverty'].fillna('<= $75,000, Above Poverty')

In [31]:
X = df

#Get Binary Data for Categorical Variables
cat_df = X[categorical_columns]
recat_df = pd.get_dummies(data=cat_df).columns

num_df = X[numerical_columns]

from sklearn.preprocessing import StandardScaler

#Scale Numerical Data
scaler = StandardScaler()
scaled_num = scaler.fit_transform(num_df)
scaled_num_df = pd.DataFrame(scaled_num, index=num_df.index, columns=num_df.columns)

encoded_df = pd.concat([num_df, scaled_num_df], axis=1)

X = np.asarray(encoded_df)

In [32]:
y = model.predict(X)
y_df = pd.DataFrame(y, columns=['h1n1_vaccine', 'seasonal_vaccine'])

In [33]:
results = pd.concat([df_full, y_df], axis=1)

In [34]:
results = results[['respondent_id', 'h1n1_vaccine', 'seasonal_vaccine']]

In [36]:
results.to_csv('../Submissions/Submission 6.6.21.csv', index=False)